In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install transformers[torch]
!pip -q install accelerate>=0.20.1
!pip install datasets
# !pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

from datasets import load_dataset

raw_datasets = load_dataset("orkg/SciQA")
train = raw_datasets.get("train")
validate = raw_datasets.get("validate")
test =  raw_datasets.get("test")
print(raw_datasets)


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def get_text():
    text = ""
    for q in train:
        query = clean(q["query"]["sparql"])
        question = q["question"]["string"]
        text += "[input] " + question
        text += "\n[output] " + clean(query)
        text += "\n\n"
    with open('train.txt', 'w', encoding="utf-8") as f:
        f.write(text)

def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)
    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=2,
        save_steps=10_000,
        save_total_limit=2,
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

# Fine-tune the mode
fine_tune_gpt2("gpt2-large", "train2.txt", "ep20")
# get_text()

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,0.400500
1000,0.248800
1500,0.187700
2000,0.161800
2500,0.153300
3000,0.135200
3500,0.126000
4000,0.123300
4500,0.108000
5000,0.111100
